<a href="https://colab.research.google.com/github/AkshathaBolla/ANN/blob/main/TitanMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module I: Practice Exercise - 'Titan Mining Corp' (Refined Edition)

### **Context: The Lunar Excavation**
You are the Chief AI Geologist for 'Titan Mining Corp'. Automated drills are extracting core samples from Saturn's moon.
Your sensors measure:
1.  **Rock Density (g/cm³)**
2.  **Radiation Level (mSv)**
3.  **Magnetic Resonance (0-10)**

You have **two separate goals**:
* **Goal A (Discovery):** Predict if the sample contains **'Unobtanium' (1)** or just **'Dirt' (0)**. (Classification)
* **Goal B (Appraisal):** Predict the **Market Value ($)** of the sample. (Regression)

---
**INSTRUCTIONS:**
This is a **High-Friction Practice**.
You must write the code to match the **Specific Architectures** provided below.
Precision matters. If the blueprint asks for `Tanh`, do not use `ReLU`.

In [3]:
# CELL 1: DATA GENERATION (Run this first)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(42)

# 200 Core Samples
N = 200
# Features: Density (2-10), Radiation (0-50), Resonance (0-10)
X = torch.rand(N, 3) * torch.tensor([8, 50, 10]) + torch.tensor([2, 0, 0])

# Target A: Is Unobtanium? (Classification)
# Rare: High Density AND High Resonance
rare_score = (X[:, 0] * 2) + (X[:, 2] * 3) - (X[:, 1] * 0.5)
y_discovery = (rare_score > 25).float().view(-1, 1)

# Target B: Market Value (Regression)
# Value = Density^2 + Resonance * 10
y_value = (X[:, 0] ** 2) + (X[:, 2] * 10) + torch.randn(N) * 5
y_value = y_value.view(-1, 1).float()

print(f"Titan Data Ready. X: {X.shape}, y_discovery: {y_discovery.shape}, y_value: {y_value.shape}")

Titan Data Ready. X: torch.Size([200, 3]), y_discovery: torch.Size([200, 1]), y_value: torch.Size([200, 1])


## **Level 1: The Mineral Scanner (User Level)**
**Your Task:** Build a model to classify samples as 'Unobtanium' or 'Dirt'.

**Blueprint (Architecture):**
1.  **Input Layer:** 3 Features
2.  **Hidden Layer:** 32 Neurons, `Tanh` Activation
3.  **Hidden Layer:** 16 Neurons, `Tanh` Activation
4.  **Output Layer:** 1 Neuron, `Sigmoid` Activation

**Training Specs:**
* Loss: `BCELoss`
* Optimizer: `Adam` (lr=0.01)
* Epochs: 150

In [4]:
# LEVEL 1: WRITE YOUR CODE HERE

# 1. Define Model Class 'MineralScanner'
class MineralScanner(nn.Module):
    # TODO: Implement __init__ with Linear(3,32) -> Tanh -> Linear(32,16) -> Tanh -> Linear(16,1) -> Sigmoid
    def __init__(self):
        super(MineralScanner, self).__init__()
        self.layer1=nn.Linear(3,32)
        self.layer2=nn.Tanh()
        self.layer3=nn.Linear(32,16)
        self.layer4=nn.Tanh()
        self.layer5=nn.Linear(16,1)
        self.layer6=nn.Sigmoid()
    def forward(self,x):
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.layer5(x)
        x=self.layer6(x)
        return x
model_scan = MineralScanner() # Initialize it

# 2. Optimizer & Loss
criterion=nn.BCELoss()
optimizer=optim.Adam(model_scan.parameters(),lr=0.01)

# 3. Training Loop (150 epochs)
for epoch in range(150):
  optimizer.zero_grad()
  outputs=model_scan(X)
  loss=criterion(outputs,y_discovery)
  loss.backward()
  optimizer.step()

# 4. Calculate Accuracy
with torch.no_grad():
  outputs=model_scan(X)
  predicted=(outputs>0.5).float()
  accuracy=(predicted==y_discovery).float().mean()
  print(f"Accuracy: {accuracy.item()}")

Accuracy: 0.9950000047683716


In [5]:
# TEST LEVEL 1
try:
    assert isinstance(model_scan, nn.Module), "Model not initialized"
    # Check Architecture
    modules = list(model_scan.children())
    # Allow for Sequential or direct list
    if isinstance(modules[0], nn.Sequential): modules = list(modules[0])

    assert modules[0].out_features == 32, f"Layer 1 must be 32. Found {modules[0].out_features}"
    assert isinstance(modules[1], nn.Tanh), "Activation 1 must be Tanh"
    assert modules[2].out_features == 16, f"Layer 2 must be 16. Found {modules[2].out_features}"

    print("✅ Level 1 Passed: Architecture matches blueprint.")
except Exception as e:
    print(f"❌ Level 1 Fail: {e}")

✅ Level 1 Passed: Architecture matches blueprint.


## **Level 2: The Value Appraiser (Overfitting & DataLoaders)**
**Your Task:** Predict the exact Market Value (Regression). You must deliberately overfit the model, then fix it using Dropout.

**Step 2.1: The Pipeline**
* Split `X` and `y_value` into **Train (80%)** and **Test (20%)**.
* Create a `DataLoader` for training (Batch Size = 10). **This tests your ability to handle batches.**

**Step 2.2: The Overfit Model (High Capacity)**
* Create a model `model_overfit`:
* Input(3) -> Linear(128) -> ReLU -> Linear(128) -> ReLU -> Output(1)
* Train for 200 epochs. (You should see Training Loss go low, but Test Loss stay high).

**Step 2.3: The Fix (Dropout)**
* Create a model `model_regularized`:
* Same architecture, but add `nn.Dropout(0.4)` after every ReLU.
* This forces the model to learn robust features.

In [6]:
# LEVEL 2: WRITE YOUR CODE HERE

# 1. Split Train/Test (Manual slicing or train_test_split)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y_value,test_size=0.2,random_state=42)
# 2. DataLoader (Batch Size 10)
train_dataset=TensorDataset(X_train,y_train)
test_dataset=TensorDataset(X_test,y_test)

train_loader=DataLoader(train_dataset,batch_size=10,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=10,shuffle=False)
# 3. Define 'model_overfit' (No Dropout)
class Modeloverfit(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1=nn.Linear(3,128)
    self.relu=nn.ReLU()
    self.linear2=nn.Linear(128,128)
    self.linear3=nn.Linear(128,1)

  def forward(self,x):
    x=self.linear1(x)
    x=self.relu(x)
    x=self.linear2(x)
    x=self.relu(x)
    x=self.linear3(x)
    return x
model_overfit=Modeloverfit()
loss_func=nn.BCEWithLogitsLoss()
optimizer=optim.Adam(model_overfit.parameters(),lr=0.01)
num_epochs=200
batch_size=10
for epoch in range(num_epochs):

    model_overfit.train()
    for X_batch,y_batch in train_loader:
      y_batch=y_batch.view(-1,1)
      logits=model_overfit(X_batch)
      loss=loss_func(logits,y_batch)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()







# 4. Define 'model_regularized' (With Dropout 0.4)
class Modelregularized(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1=nn.Linear(3,128)
    self.relu=nn.ReLU()
    self.dropout=nn.Dropout(0.4)
    self.linear2=nn.Linear(128,128)
    self.linear3=nn.Linear(128,1)

  def forward(self,x):
    x=self.linear1(x)
    x=self.relu(x)
    x=self.dropout(x)
    x=self.linear2(x)
    x=self.relu(x)
    x=self.dropout(x)
    x=self.linear3(x)
    return x
model_regularized=Modelregularized()
loss_func=nn.BCEWithLogitsLoss()
optimizer=optim.Adam(model_regularized.parameters(),lr=0.01)
num_epochs=200
batch_size=10
for epoch in range(num_epochs):
    model_regularized.train()
    for X_batch,y_batch in train_loader:
      y_batch=y_batch.view(-1,1)
      logits=model_regularized(X_batch)
      loss=loss_func(logits,y_batch)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()


# 5. Optional: Train both to see the difference (Not graded, but recommended)

In [7]:
# TEST LEVEL 2
try:
    # Check Dropout
    has_dropout = any(isinstance(m, nn.Dropout) for m in model_regularized.modules())
    assert has_dropout, "Dropout layer not found in model_regularized"

    # Check Dropout Probability
    for m in model_regularized.modules():
        if isinstance(m, nn.Dropout):
            assert m.p == 0.4, f"Dropout must be 0.4, found {m.p}"

    # Check Dimensions
    l1 = list(model_regularized.modules())[1] # First Linear
    assert l1.out_features == 128, "Hidden neurons must be 128"

    print("✅ Level 2 Passed: Data Pipeline and Dropout correct.")
except Exception as e:
    print(f"❌ Level 2 Fail: {e}")

✅ Level 2 Passed: Data Pipeline and Dropout correct.


## **Level 3: The Lab Tournament (Custom Combinations)**

**Part 3.1: Manual Custom Loss**
Implement `my_weighted_mse(pred, target)`.
Sometimes, underestimating the value is worse than overestimating.
$$ Loss = Mean( (pred - target)^2 * 1.5 ) $$
(Multiply the standard squared error by 1.5).

**Part 3.2: The Tournament**
You must define the `experiments` list. Each experiment is a dictionary defining a specific combination of **Activation** and **Loss Function**.

**The Contenders:**
1.  **"Standard"**: Uses `nn.ReLU` and `nn.MSELoss`.
2.  **"Robust"**: Uses `nn.LeakyReLU` (negative_slope=0.1) and `nn.L1Loss` (MAE).
3.  **"Custom"**: Uses `nn.ELU` and **YOUR** `my_weighted_mse` function.

**Task:** Fill in the `experiments` list with the correct objects.

In [13]:
# LEVEL 3.1: MANUAL CUSTOM LOSS
def my_weighted_mse(logits, y_value):
  loss=((logits-y_value)**2)*1.5
    # TODO: Return (pred - target)^2 * 1.5
  return loss

# LEVEL 3.2: TOURNAMENT CONFIG
# TODO: Fill in the 'None' values with actual Class Instances or Functions
experiments = [
    {"name": "Standard", "act": nn.ReLU(), "loss": nn.MSELoss()}, # ReLU, MSELoss
    {"name": "Robust",   "act": nn.LeakyReLU(negative_slope=0.1), "loss": nn.L1Loss()}, # LeakyReLU(0.1), L1Loss
    {"name": "Custom",   "act": nn.ELU(), "loss": my_weighted_mse}  # ELU, my_weighted_mse
]


In [14]:
# TEST LEVEL 3
try:
    # Test Manual Loss
    p = torch.tensor([2.0]); t = torch.tensor([4.0])
    # (2-4)^2 * 1.5 = 4 * 1.5 = 6.0
    assert my_weighted_mse(p, t).item() == 6.0, "Math Fail: Weighted MSE calculation incorrect"

    # Test Configs
    assert isinstance(experiments[0]['loss'], nn.MSELoss), "Standard must use MSELoss"
    assert experiments[1]['act'].negative_slope == 0.1, "Robust must use LeakyReLU with 0.1 slope"
    assert not isinstance(experiments[2]['loss'], type), "Custom must use your function, not a class"

    print("✅ Level 3 Passed: Manual Loss and Tournament Configs are correct.")
except Exception as e:
    print(f"❌ Level 3 Fail: {e}")

✅ Level 3 Passed: Manual Loss and Tournament Configs are correct.


## **Level 4: The Core Engineer (Expert Level)**
**Your Task:** The `optim.Adam` and `loss.backward()` buttons are broken.
Perform **One Step of Gradient Descent** manually.

1.  Model: $y = w \cdot x$ (No bias for simplicity).
2.  Input $x=2.0$, Target $y=10.0$.
3.  Initial Weight $w=3.0$.
4.  Loss Function: Squared Error $L = (wx - y)^2$.
5.  **Calculate Gradient:** $\frac{dL}{dw} = 2(wx - y) \cdot x$.
6.  **Update Weight:** $w_{new} = w - lr \cdot gradient$.

In [15]:
# LEVEL 4: THE REPAIR JOB
lr = 0.1
x = 2.0
y = 10.0
w = 3.0

# TODO: Calculate prediction
pred = w*x

# TODO: Calculate squared error
loss = pred-y

# TODO: Calculate gradient manually (Derived from Chain Rule)
gradient = 2*loss*x

# TODO: Update w
w_new = w-(lr*gradient)

In [16]:
# TEST LEVEL 4
try:
    # Pred = 3*2 = 6
    # Error = (6-10) = -4
    # Grad = 2 * (-4) * 2 = -16
    # w_new = 3.0 - (0.1 * -16) = 3.0 + 1.6 = 4.6

    assert abs(w_new - 4.6) < 0.01, f"Math Fail: New weight should be 4.6, got {w_new}"
    print("✅ Level 4 Passed: Manual Backprop successful!")
except Exception as e:
    print(f"❌ Level 4 Fail: {e}")

✅ Level 4 Passed: Manual Backprop successful!
